# Planet Destinations Python Client Introduction

This tutorial is an introduction to [Planet's](https://www.planet.com) Destination API using the official [Python client](https://github.com/planetlabs/planet-client-python), the `Planet` module. The Destination API allows you manage and securely store cloud storage bucket credentials as a 'Destination' for a product delivery on the Planet Platform. This can serve as an alternative to adding credentials to multiple workflows, for example, as the Destination can be set up once, and then referenced in different Order or Subscription requests.

## Requirements

An account on the [Planet Platform](https://www.planet.com/account/) is required to access any of Planet's API's. If you are not logged in, you will be prompted to do so below in the *SDK Authentication* section.

## Useful links 
* [Planet SDK for Python](https://planet-sdk-for-python.readthedocs.io/en/stable/get-started/quick-start-guide/)
* [Planet Python Client Repo](https://github.com/planetlabs/planet-client-python)
* [Planet Destinations Documentation](https://docs.planet.com/develop/apis/destinations/)

The basic workflow for interaction with the Destinations API is:
1. Prepare destination variables; mainly credentials and bucket name, with some other items depending on cloud storage provider.
2. Create Destination using the Destinations API
3. Use the Destination Reference ID in Orders and Subscription Requests.

____

## <u>Destination Information</u>

#### Supported Cloud Storage Destinations:
* Amazon S3
    * Most other services that implement the S3 compatibility API
* Google Cloud Storage
* Microsoft Azure Blob Storage
* Oracle Cloud Storage


#### Destinations Limitations:
* Each *organization* is provided a limitation of up to 50 Destinations, which will be both visible and usable by each individual within said organization.
* The creator of a Destination is considered its *owner*, and, along with your organizational administrator, will be the only one allowed to modify a Destination.
* All Destinations must have unique names.
* All Destination API requests are rate limited to 3 requests per second.

#### Destination Credentials Information
* Credentials used to create a Destination must have both `write` and `delete` permissions (the specific names of these permissions may vary).
    * When a destination is created or updated, Planet verifies access by attempting to write and then delete a test file named `planetverify.txt`. 
    * If the permissions are correctly configured, this file will be removed immediately and will not appear in the storage bucket or container.
    * If you ever see this file, your credentials may be lacking the `delete` permissions needed.

#### Credential Security
* Destination credentials are treated as secrets which are encrypted at rest and in transit between Planet systems, accessed and decrypted only when strictly required, during a delivery.
* Destinations API will always redact these secrets in its responses.
    * You can update credentials via the Destination API, but never read them using it.


____

## Set up

In order to interact with the Planet Destinations API using the Python client, we need to import the necessary packages and authenticate our Planet account credentials.

### Imports

In [ ]:
import base64
import planet
import getpass
from datetime import datetime
from planet import data_filter, order_request, reporting, subscription_request

### SDK Authentication

Your Planet login is used to authenticate and activate the Python SDK. You will be prompted via a link below to login and confirm on the page that the code displayed matches the authorization code printed. If this is your first time accessing the Planet SDK, you will also be prompted first to authorize the SDK access to your Planet account. 
If you would like to know more, please visit the [authentication documentation](https://docs.planet.com/develop/authentication).

In [40]:
# Authenticate for the Planet SDK; See docs: https://docs.planet.com/develop/authentication
# If you are not already logged in, this will prompt you to open a web browser to log in.

auth = planet.Auth.from_profile('planet-user', save_state_to_storage=True)
auth.ensure_initialized(allow_open_browser=False, allow_tty_prompt=True)

session = planet.Session(auth)
pl = planet.Planet(session)

_____

### Prepare Destination Credentials

Before you create a Destination, you must first complete the formation of a credentials dictionary. While this has slight variations between cloud provider, this will always have a `name`, a `type` (provider), with `parameters` that will contain the `bucket name` and some sort of `credential keys`. 

For convenience, placeholders for the various supported cloud storage providers will be in the cells below. The names of the specific permissions with also be listed.

The various secret keys for credentials are meant to be placeholders, and can be set as environment variables to avoid pasting them in anywhere. Alternatively, they can be entered using `getpass` below:

In [41]:
# Use this variable in place of things like aws_secret_access_key, credentials_str, sas_token, etc.
credential_secret = getpass.getpass('credential secret:')

#### <u>Amazon S3</u>
For Amazon S3 delivery use an AWS account with `GetObject`, `PutObject`, and `DeleteObject` permissions.

You will also need a `aws_region`, `aws_access_key_id`, and `aws_secret_access_key`.

In [ ]:
name = 's3 destination'
bucket = 'bucket name'
aws_region = 'region name'
aws_access_key_id = 'access key id'
aws_secret_access_key = 'aws secret access key' # or credential_secret variable

bucket_creds = {
    "name": name,
    "type": "amazon_s3",
    "parameters": {
        "bucket": bucket,
        "aws_region": aws_region,
        "aws_access_key_id": aws_access_key_id,
        "aws_secret_access_key": aws_secret_access_key,
    },
}

#### <u> Google Cloud Storage </u>

For Google Cloud Storage delivery, a service account with `storage.objects.create`, `storage.objects.get`, and `storage.objects.delete` permissions is required.

***Important***: The Google Cloud Storage delivery option requires a single-line base64 version of the service account credentials for use by the credentials parameter.

Download the service account credentials in JSON format (not P12) and use the following code with the `base64` module to encode the json as a string from a path:

In [ ]:
json_key_path = "JSON FILE PATH"
with open(json_key_path, "rb") as f:
        credentials_str = base64.b64encode(f.read()).decode()

In [ ]:
name = 'GCS destination'
bucket_name = 'bucket name'

bucket_creds = {
    "name": name,
    "type": "google_cloud_storage",
    "parameters": {
        "bucket": bucket_name,
        "credentials": credentials_str,
    },
}

#### <u> Microsoft Azure Blob Storage </u>

For Microsoft Azure delivery use an Azure account with `read`, `write`, `delete`, and `list` permissions.

You will also need: `account name`, `container_name`, and `sas_token`.

In [ ]:
name = 'azure destination'
account = 'account name'
container_name = 'container name'
sas_token = 'token' # or credential_secret variable

# Optional: storage endpoint suffix:
# storage_endpoint_suffix = 'storage endpoint suffix'

bucket_creds = {
    "name": name,
    "type": "azure_blob_storage",
    "parameters": {
        "account": account,
        "container": container_name,
        "sas_token": sas_token,
        #   "storage_endpoint_suffix": storage_endpoint_suffix, # optional
    },
}

#### <u> Oracle Cloud Storage </u>
For Oracle Cloud Storage delivery, use an Oracle account with `read`, `write`, and `delete` permissions.

For authentication, use a `Customer Secret Key` which consists of an `Access Key/Secret Key` pair.

In [ ]:
name = 'ocs name'
bucket_name = 'bucket name'
region = 'region'
namespace = 'namespace'
customer_access_key_id = 'customer_access_key_id'
customer_secret_key = 'customer_secret_key' # or credential_secret variable

bucket_creds = {
    "name": name,
    "type": "oracle_cloud_storage",
    "parameters": {
        "bucket": bucket_name,
        "region": region,
        "namespace": namespace,
        "customer_access_key_id": customer_access_key_id,
        "customer_secret_key": customer_secret_key,
    },
}

#### <u> Other S3 Compatible Storage Providers </u>


Other cloud hosting services that implement the S3 compatability API are allowed, with similar schema to the normal AWS delivery. To use this delivery method, use an account with `read`, `write`, and `delete` permissions for the bucket.

Authentication is performed using an Access Key and Secret Key pair.

You will also need `endpoint`, and can optionally use a path style.
* Pay attention to the `use_path_style` parameter if you choose a path style, as it is a common source of issues.
    * For example, Oracle Cloud requires use_path_style to be true, while Open Telekom Cloud requires it to be false.

In [ ]:
name = 's3 compatible destination'
bucket = 'bucket name'
region = 'region name'
endpoint = 'endpoint'
access_key_id = 'access key id'
secret_access_key = 'secret access key' # or credential_secret variable

# Optional:
# use_path_style = False

bucket_creds = {
    "name": name,
    "type": "s3_compatible",
    "parameters": {
        "bucket": bucket,
        "region": region,
        "endpoint": endpoint,
        "access_key_id": access_key_id,
        "secret_access_key": secret_access_key,
        # "use_path_style": use_path_style
    },
}

___

### Destination Creation

Once you have set up your credentials dictionary above, you should use it to create a Destination:

In [ ]:
new_destination = pl.destinations.create_destination(bucket_creds)

In [ ]:
new_destination

{'_links': {'_self': 'https://api.planet.com/destinations/v1/demo-destination-5aIp1Kiqd76pOK4lWDy1Y'},
 'archived': None,
 'created': '2026-02-12T18:41:22.659959594Z',
 'default': False,
 'id': 'demo-destination-5aIp1Kiqd76pOK4lWDy1Y',
 'name': 'demo destination',
 'ownership': {'is_owner': True, 'owner_id': '<owner_id>'},
 'parameters': {'bucket': 'simon-test-bucket-pl', 'credentials': '<REDACTED>'},
 'permissions': {'can_write': True},
 'pl:ref': 'pl:destinations/demo-destination-5aIp1Kiqd76pOK4lWDy1Y',
 'type': 'google_cloud_storage',
 'updated': '2026-02-12T18:41:22.659960762Z'}

_____

### Destination ID and Reference ID's

Once you have created your Destination, we will interact with it using its Reference ID, stored as the 'pl:ref' key. We will also save the ID of destination, stored as the 'id' key.

While it is confusing why we have both variables, you will notice that the Destination *Reference* ID contains the ID of the destination, with the added `pl:destinations/` string. 

The Reference ID is used to reference a destination in other functions and API's, like Orders and Subscriptions. If you need to access or change any of the information held within a Destination, you will use the Destination ID (not the reference ID), using `pl.destinations.get_destination` and `pl.destination.patch_destination` respectively.

In [ ]:
dest_ref_id = new_destination['pl:ref']
destination_id = new_destination['id']

print(f'Destination id: {destination_id}')
print(f'Destination Reference ID: {dest_ref_id}')

Destination id: demo-destination-5aIp1Kiqd76pOK4lWDy1Y
Destination Reference ID: pl:destinations/demo-destination-5aIp1Kiqd76pOK4lWDy1Y


### Other Destinations API Functions:

* Listing Destinations
* Modify a Destination
    * *Only Authentication parameters may be modified for a destination. Bucket names, regions, etc, may not be changed.*
* Setting a Default Destination
* Unset a Default Destination
    * *All users in an organization may access the Default Destination, but only the admin may set and unset it*


In [ ]:
all_destinations = pl.destinations.list_destinations()
print(f'You have access to {len(all_destinations['destinations'])} destinations.')

all_destinations

You have access to 1 destinations.


#### Access a specific Destination

If you have a `Destination_id` but need the rest of the key value pairs belonging to it, use `get_destination.`

In [ ]:
pl.destinations.get_destination(destination_id)

#### Patching a Destination

If you make changes to your bucket's access credentials, you can use `patch_destination` to update them. You can **only** update credentials/secrets with this method.

We will not be listing every method every cloud provider here, but you will need to change the `credential` key to match your provider, `aws_secret_access_key` for AWS, `sas_token` for Azure, etc.

In [ ]:
new_credentials = ''

dest_param_to_update = {
    'credentials' : new_credentials
}

pl.destinations.patch_destination(destination_id, dest_param_to_update)

_____

#### Set your Default Destination *For Organizational Admins Only*

Setting your Default Destination is a quick way to speed up delivery with Orders and Subscriptions with the Destinations API. Note that changing the Default Destination is a power *only available to an organization's admin account*. The following functions will not be available at all in the SDK if you are not your organization's admin. If you are not the admin, feel free to skip this section of the tutorial. 



In [ ]:
default_destination = pl.destinations.set_default_destination(dest_ref_id)
default_destination

{'_links': {'_self': 'https://api.planet.com/destinations/v1/demo-destination-5aIp1Kiqd76pOK4lWDy1Y'},
 'archived': None,
 'created': '2026-02-12T18:41:22.659959594Z',
 'default': True,
 'id': 'demo-destination-5aIp1Kiqd76pOK4lWDy1Y',
 'name': 'demo destination',
 'ownership': {'is_owner': True, 'owner_id': '<owner_id>'},
 'parameters': {'bucket': 'simon-test-bucket-pl', 'credentials': '<REDACTED>'},
 'permissions': {'can_write': True},
 'pl:ref': 'pl:destinations/demo-destination-5aIp1Kiqd76pOK4lWDy1Y',
 'type': 'google_cloud_storage',
 'updated': '2026-02-12T18:41:22.659960762Z'}

You can also 'unset' a destination from the default. This requires no input and will have no output:

In [ ]:
## Uncomment this if you need to unset your default destination.
# pl.destinations.unset_default_destination()

Once the default destination is set, it can be used in the `delivery` input in Orders and Subscriptions. If you set your default destination above, you don't need to run this cell, as the setting the default_destination will have the same output as getting it. However we are providing it as a template.

In [11]:
default_destination = pl.destinations.get_default_destination()

____

### Using the Destination in other API calls.

Once you have a Destination set up, you can use its Reference ID as the `delivery` argument for Orders and Subscriptions. There are some handy functions for setting a `destination` or `default_destination` from both the `order_request` and `subscription_request` submodules that we will use.

 We will reuse the variable `dest_ref_id` we created before to demonstrate this by creating a Data API search request for imagery of San Francisco, and Order a PlanetScope Scene that will be clipped and delivered to our new Destination. Using a destination in a Subscription will follow a very similar pattern.

*WARNING* - Ordering Imagery like this will consume some of your quota. If you wish to follow this demo anyways, feel free to change the parameters to an AOI/TOI/asset types/ etc. that will be useful and relevant to you. 

#### Order Requests:

In [ ]:
destination_dict = order_request.destination(dest_ref_id)

# Use this instead to use a default_destination:

# destination_dict = order_request.default_destination()

#### Subscription Requests:

In [ ]:
destination_dict = subscription_request.destination(dest_ref_id)

# Use this instead to use a default_destination:

# destination_dict = subscription_request.default_destination()

In [48]:
# San Francisco Geometry

geom = {'type': 'Polygon',
 'coordinates': [[[-122.47455596923828, 37.810326435534755],
   [-122.49172210693358, 37.795406713958236],
   [-122.52056121826172, 37.784282779035216],
   [-122.51953124999999, 37.6971326434885],
   [-122.38941192626953, 37.69441603823106],
   [-122.38872528076173, 37.705010235842614],
   [-122.36228942871092, 37.70935613533687],
   [-122.34992980957031, 37.727280276860036],
   [-122.37773895263672, 37.76230130281876],
   [-122.38494873046875, 37.794592824285104],
   [-122.40554809570311, 37.813310018173155],
   [-122.46150970458983, 37.805715207044685],
   [-122.47455596923828, 37.810326435534755]]]}

In [49]:
# Define the filters we'll use to find our data

item_types = ["PSScene"]
limit = 50

# Filters for our search request
geom_filter = data_filter.geometry_filter(geom)
clear_percent_filter = data_filter.range_filter('clear_percent', 90)
date_range_filter = data_filter.date_range_filter("acquired", gt=datetime(month=1, day=1, year=2026))
cloud_cover_filter = data_filter.range_filter('cloud_cover', None, 0.1)

combined_filter = data_filter.and_filter([geom_filter, clear_percent_filter, date_range_filter, cloud_cover_filter])

search_request = pl.data.create_search(item_types=item_types, search_filter=combined_filter, name='planet_sdk_destination_demo')


# Search the Data API
search_id = search_request['id']
item_list = pl.data.run_search(search_id=search_id, limit=limit)
item_list = list(item_list)
item_id = item_list[0]['id']

In [50]:
# Define request details to properly build your request

item_ids = [item_id]

products = [
    order_request.product(item_ids, 'analytic_udm2', 'PSScene')
]

tools = [
    order_request.reproject_tool(projection='EPSG:4326', kernel='cubic'),
    order_request.clip_tool(geom)
]

order_built = order_request.build_request(
    'destination_test_order', products=products, tools=tools, delivery=destination_dict)


In [36]:
order_built

{'name': 'destination_test_order',
 'products': [{'item_ids': ['20260212_192702_52_254a'],
   'item_type': 'PSScene',
   'product_bundle': 'analytic_udm2'}],
 'delivery': {'destination': {'ref': 'pl:destinations/demo-destination-5aIp1Kiqd76pOK4lWDy1Y'}},
 'tools': [{'reproject': {'projection': 'EPSG:4326', 'kernel': 'cubic'}},
  {'clip': {'aoi': {'type': 'Polygon',
     'coordinates': [[[-122.47455596923828, 37.810326435534755],
       [-122.49172210693358, 37.795406713958236],
       [-122.52056121826172, 37.784282779035216],
       [-122.51953124999999, 37.6971326434885],
       [-122.38941192626953, 37.69441603823106],
       [-122.38872528076173, 37.705010235842614],
       [-122.36228942871092, 37.70935613533687],
       [-122.34992980957031, 37.727280276860036],
       [-122.37773895263672, 37.76230130281876],
       [-122.38494873046875, 37.794592824285104],
       [-122.40554809570311, 37.813310018173155],
       [-122.46150970458983, 37.805715207044685],
       [-122.474555969

In [52]:
created_order = pl.orders.create_order(order_built)
order_id = created_order['id']
print(f'New Order Created: {order_id}')

with reporting.StateBar() as bar:
    pl.orders.wait(order_id, callback=bar.update_state, delay=10,max_attempts=500)

New Order Created: 9747430b-0886-4bf7-a7ef-33d1bd33fd8c


05:30 - order  - state: success


____

Congratulations! You have completed the Destinations API demo! You now have the tools to set up destinations for your organizations, which make your product deliveries more secure. Feel free to use your Destinations in any future Orders and Subscriptions that you create.